Networking Two Docker Containers
------------------------------

Docker makes it incredibly simple to build a network of containers on a device. We'll take advantage of this today to create a data serving container, and a container that processes our streams.

We'll need a terminal or powershell session to manage the data server, and we'll interact with our stream processing container through our typical Jupyter notebook.

Getting Going
------------

Before we start containers, we need to set up our network. As you might expect, Docker's syntax for interacting with networks builds on the syntax we're already familiar with. We've seen `docker image`, `docker container`, and `docker build`. The command for interacting with Docker networks, then, is simply: `docker network`.

We'll need to do the following:
1. Create our network.
2. Start our containers and attach them to the network.
3. Inspect the network to make sure that the two containers are, in fact, properly attached.
4. Stream data across the network.

If you want to learn more about networking with Docker, the best place to start is with their [network tutorial](https://docs.docker.com/network/network-tutorial-standalone/).

To create our network, we simply use the following command in a terminal window: `docker network create --driver bridge thinkful-net`. This command uses the default bridge network and creates a network called `thinkful-net`.

To confirm that the network was set up, we can check with: `docker network ls`.

Now it's time to launch our two containers. We'll use the standard `docker run` command, but in this case we have two containers, not one. We want them to connect to the network, have a readable name, and map the right data into each container. In addition, for the data server, we need to easily get into the container's shell so we can activate the server.

Launch the server with the following command:
`docker run -dit --rm --name data_server --network thinkful-net -v /Users/damian/Documents/Code/ds-notebooks/Spark/StructuredStreaming/ServeStreams:/home/ds/data thinkfulstudent/simple_server /bin/bash`

Note the switches we use:
* `-dit` launches in the background, but also lets the container know we may want an interactive session.
* `--rm` removes the container when we stop it.
* `--name` allows us to assign our own name to the container.
* `--network` attaches to the network.
* `-v` is the same folder mapping we've consistently used to assign data to the container.

After the image name (`thinkfulstudent/simple_server`), you'll note that we have a `/bin/bash` statement - that's so we can easily get into our shell.

If you want to see that in action, type: `docker attach data_server` and you'll get a shell. Exit the shell with the shortcut `ctrl-p ctrl-q`. We'll come back here later to launch the server.

Launch the Spark instance in the same way we have before, but this time we'll also add a name and a network flag to the run statement.

`docker run -d --rm --name pyspark1 --network thinkful-net -v /Users/damian/Documents/Code/ds-notebooks/Spark/StructuredStreaming/ProcessStreams:/home/ds/notebooks -p 8888:8888 thinkfulstudent/pyspark:2.2.1`

Once both containers are up and running, we need to confirm that they are on the `thinkful-net` network, and find their IP addresses so we can have them talk to each other.

Check the network status by typing `docker network inspect thinkful-net`. You should see formatted json output to the window, and if you find the `Containers` field, you'll see the name and IPv4 address of each container. Take note of these, we may need them later.

Now that the network is set up and our containers are running, it's time to get going on our streaming datasets.